# GEOGRAPHIC PARTS
# IMPORT LIBS

In [13]:
import pandas as pd
import plotly.express as px
import math
import numpy as np
import geopy.distance

# OPEN DATASET

In [14]:
df = pd.read_csv('../../DATA/PREPROCESSING/prepro_00_dropna.csv')
df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...,...
1953457,2014-11-17 13:51:02.0000002,4.0,2014-11-17 13:51:02 UTC,-73.950787,40.770836,-73.951043,40.774543,1
1953458,2013-02-25 17:51:28.0000004,7.0,2013-02-25 17:51:28 UTC,-73.983035,40.748276,-73.974725,40.741978,1
1953459,2013-07-12 22:03:41.0000006,10.5,2013-07-12 22:03:41 UTC,-74.003482,40.743577,-73.976758,40.765265,1
1953460,2009-01-17 09:35:37.0000002,10.9,2009-01-17 09:35:37 UTC,-73.971819,40.797243,-73.985921,40.752459,3


# DELETE ALL LATITUDE OUT OF RANGE

In [15]:
df = df.loc[(df['pickup_latitude'] >= 40 ) & (df['pickup_latitude'] <= 42 ) & (df['dropoff_latitude'] >= 40 ) & (df['dropoff_latitude'] <= 42 ) & (df['pickup_longitude'] >= -76 ) & (df['pickup_longitude'] <= -72 ) & (df['dropoff_longitude'] >= -76 ) & (df['dropoff_longitude'] <= -72 )]
df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...,...
1953457,2014-11-17 13:51:02.0000002,4.0,2014-11-17 13:51:02 UTC,-73.950787,40.770836,-73.951043,40.774543,1
1953458,2013-02-25 17:51:28.0000004,7.0,2013-02-25 17:51:28 UTC,-73.983035,40.748276,-73.974725,40.741978,1
1953459,2013-07-12 22:03:41.0000006,10.5,2013-07-12 22:03:41 UTC,-74.003482,40.743577,-73.976758,40.765265,1
1953460,2009-01-17 09:35:37.0000002,10.9,2009-01-17 09:35:37 UTC,-73.971819,40.797243,-73.985921,40.752459,3


# CALCUL DISTANCE WITH GEOPY LIB

In [16]:
df['Distance (miles)'] = df.apply(lambda row: geopy.distance.geodesic((row['pickup_latitude'], row['pickup_longitude']), (row['dropoff_latitude'], row['dropoff_longitude'])).miles, axis=1)

df

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_30320\1709050520.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,Distance (miles)
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1,0.639764
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1,5.246511
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2,0.863167
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1,1.737223
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1,1.241710
...,...,...,...,...,...,...,...,...,...
1953457,2014-11-17 13:51:02.0000002,4.0,2014-11-17 13:51:02 UTC,-73.950787,40.770836,-73.951043,40.774543,1,0.256146
1953458,2013-02-25 17:51:28.0000004,7.0,2013-02-25 17:51:28 UTC,-73.983035,40.748276,-73.974725,40.741978,1,0.615670
1953459,2013-07-12 22:03:41.0000006,10.5,2013-07-12 22:03:41 UTC,-74.003482,40.743577,-73.976758,40.765265,1,2.050854
1953460,2009-01-17 09:35:37.0000002,10.9,2009-01-17 09:35:37 UTC,-73.971819,40.797243,-73.985921,40.752459,3,3.177537


On this plot, we can see too many drive with high distance but cheap price AND too many drive with cheap distance but high price.
So, we gonna limit this by adding a price by kmfeature and try to drpo rows with price by km useless

# VISUALISATION OF OUTLIERS
## DISPLAY DISTANCE DISTRIBUTION

In [17]:
fig = px.violin(df, y="Distance", box=True,
                # points="all",
                # hover_data=df.columns,
                title='Violin plot of Distance distribution'
                )
fig.show()

ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count', 'Distance (miles)'] but received: Distance

# Calcul D’Outliers :
X < q1 - 1.5 x iqr X > q3 - 1.5 x iqr

In [ ]:
q1 = df['Distance'].quantile([.25])
q1 = q1[0.25]
q3 = df['Distance'].quantile([.75])
q3 = q3[0.75]

iqr = q3 - q1

In [ ]:
print(f" IQR = {iqr}, q1 = {q1}, q3 = {q3}")

In [ ]:
outliers_superieur = q3 + (1.5 * iqr)
outliers_inferieur = q1 - (1.5 * iqr)

print(f'Outliers Inférieur = {outliers_inferieur}')
print(f'Outliers Supérieur = {outliers_superieur}')

# DROP OUTLIERS

In [ ]:
df = df.loc[(df['Distance'] <= outliers_superieur) & (df['Distance'] >= outliers_inferieur)]
df

# DISPLAY NEW VIOLIN PLOT WITHOUT OUTLIERS

In [ ]:
fig = px.violin(df, y="Distance", box=True,
                # points="all",
                # hover_data=df.columns,
                title='Violin plot of Distance distribution'
                )
fig.show()

# SCATTER PLOT OF FARE AMOUNT BY DISTANCE

In [ ]:
fig = px.scatter(df, x='Distance', y='fare_amount', color='passenger_count')
fig.show()

# EXPORT CSV

In [ ]:
df.to_csv('../../DATA/PREPROCESSING/prepro_01_geo.csv', index=False)